In [4]:
import pandas as pd

csv_files = ['C:/Users/raghul/Desktop/redbus project/APSRTC.csv',
             'C:/Users/raghul/Desktop/redbus project/ASTC.csv',
             'C:/Users/raghul/Desktop/redbus project/CTU_RTC.csv',
             'C:/Users/raghul/Desktop/redbus project/HRCT.csv',
             'C:/Users/raghul/Desktop/redbus project/KERALA_RTC.csv',
             'C:/Users/raghul/Desktop/redbus project/KTCL.csv',
             'C:/Users/raghul/Desktop/redbus project/PEFSU.csv',
             'C:/Users/raghul/Desktop/redbus project/SBSTC.csv',
             'C:/Users/raghul/Desktop/redbus project/TSRTC.csv',
             'C:/Users/raghul/Desktop/redbus project/WBTC.csv']

data = pd.read_csv(csv_files[0])

for file in csv_files[1:]:
    temp_df = pd.read_csv(file)
    data = pd.concat([data, temp_df], ignore_index=True)
data = data.dropna()

data.to_csv('FullData_output.csv', index=False)

print("CSV files merged successfully!")


CSV files merged successfully!


In [7]:
print(data.columns)

Index(['State', 'Route', 'Bus Name', 'Bus Type', 'Departing Time',
       'Duration Time', 'Reaching Time', 'Price', 'Seats Available',
       'Star Rating', 'state', 'Route ', 'Route Name'],
      dtype='object')


In [8]:
columns_to_remove = ['Route ', 'Route Name']
data = data.drop(columns=columns_to_remove, errors='ignore') 

In [5]:
data

,State,Route,Bus Name,Bus Type,Departing Time,Duration Time,Reaching Time,Price,Seats Available,Star Rating
0,APSRIC,Hyderabad to Vijayawada,FRESHBUS,Electric A/C Seater (2+2),04:35,05h 55m,10:30,1998.0,23 Seats available,4.7
1,APSRIC,Hyderabad to Vijayawada,FRESHBUS,Electric A/C Seater (2+2),22:55,06h 15m,05:10,2298.0,15 Seats available,4.6
2,APSRIC,Hyderabad to Vijayawada,FRESHBUS,Electric A/C Seater (2+2),05:50,05h 55m,11:45,1998.0,42 Seats available,4.6
3,APSRIC,Hyderabad to Vijayawada,AdSitara Travels,NON A/C Push Back (2+2),22:40,05h 20m,04:00,944.0,17 Seats available,4.6
4,APSRIC,Hyderabad to Vijayawada,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),23:50,05h 45m,05:35,949.0,25 Seats available,4.4
...,...,...,...,...,...,...,...,...,...,...
6340,WBTC,Kolkata to Digha,Sagufta Travels (Mayuri),A/C Seater (2+2),23:15,04h 45m,04:00,515.0,21 Seats available,3.6
6341,WBTC,Kolkata to Digha,Arunita Paribahan,AC Seater (2+2),23:45,05h 00m,04:45,670.0,17 Seats available,5.0
6342,WBTC,Kolkata to Digha,Sagufta Travels(Maa Chandi),A/C Seater / Sleeper (2+2),22:10,07h 30m,05:40,515.0,30 Seats available,3.2
6343,WBTC,Kolkata to Digha,Parameswar Travels,A/C Seater Push Back (2+3),21:50,05h 55m,03:45,400.0,8 Seats available,3.8


In [10]:

data.to_csv('FullData_output.csv', index=False)

In [1]:
import psycopg2
import pandas as pd
import re

connection = psycopg2.connect(
    host="localhost",
    database="redbus_database",
    user="postgres",
    password="1234"  
)
cursor = connection.cursor()

create_table_query = '''
CREATE TABLE IF NOT EXISTS redbus_data (
    ID SERIAL PRIMARY KEY,
    state VARCHAR(50),
    route VARCHAR(100),
    bus_name VARCHAR(100),
    bus_type VARCHAR(100),
    departing_time VARCHAR(50),
    duration_time VARCHAR(50),
    reaching_time VARCHAR(50),
    price FLOAT,
    seats_available INT,
    star_rating FLOAT
);
'''
cursor.execute(create_table_query)
connection.commit()


csv_file_path = 'C:/Users/raghul/Desktop/redbus project/FullData_output.csv' 
df = pd.read_csv(csv_file_path)

df['Seats Available'] = df['Seats Available'].apply(lambda x: int(re.sub(r'\D', '', str(x))) if pd.notnull(x) else 0)


df = df.fillna('') 

insert_query = '''
INSERT INTO redbus_data (state, route, bus_name, bus_type, departing_time, duration_time, reaching_time, price, seats_available, star_rating)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
'''


cursor.executemany(insert_query, df.values.tolist())

connection.commit()


cursor.close()
connection.close()

print("Data has been inserted into the table successfully.")


Data has been inserted into the table successfully.
